# PyTorch를 이용한 경사 하강법과 선형 회귀

### "Deep Learning with Pytorch: Zero to GANs"의 2번째 파트

이 튜토리얼은 [PyTorch](https://pytorch.org)를 이용한 초보자용 딥러닝 학습 튜토리얼 입니다.   
학습하기 위한 최고의 방법은 본인이 코드를 실행하고, 실험해 보는 것이기 때문에 이 튜토리얼은 실용성과 코딩 중심으로 진행됩니다.

이번 튜토리얼에선 다음과 같은 주제를 다룹니다:

- 선형 회귀와 경사 하강법의 소개
- 선형 회귀 모델을 PyTorch의 텐서를 사용한 구현
- 경사 하강법을 이용한 선형 회귀 모델 학습
- PyTorch 기본 함수를 이용한 경사 하강법과 선형 회귀 구현

시작하기 전, 필요한 라이브러리들을 설치해야만 합니다.<br>
PyTorch의 설치는 여러분의 실행 시스템/클라우드 환경에 따라 다릅니다.<br>
자세한 설치 명령어의 내용은 다음 주소에서 확인 할 수 있습니다: https://pytorch.org

In [56]:
# 필요한 경우 주석 처리를 제거하고 여러분의 운영 체제에 적합한 명령을 실행하시면 됩니다.

# Linux / Binder
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy torch torchvision torchaudio

---
## 선형 회귀

이번 튜토리얼에서 저희는 _선형 회귀_ 라는 머신러닝 알고리즘의 기초에 대해 알아볼 것입니다.<br>
저희는 각 지역의 평균 기온, 강수량, 습도 (_입력 변수_)를 보고 사과와 오렌지의 수확량(_목표 변수_)을 예측하는 모델을 만들 것입니다.<br>
아래 그림은 학습 데이터의 일부입니다.

![선형 회귀 학습 데이터](https://i.imgur.com/6Ujttb4.png)

선형 회귀 모델에서, 각 목표 변수는 입력 변수의 <font color=red>가중 합계(weighted sum)</font>로 추정되고 편향(bias)이라는 변수로 상쇄됩니다 : 

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

아래에 있는 전체 학습 데이터로 학습한 선형 회귀 그래프를 보면 사과의 수확량이 온도, 강우 및 습도라는 변수의 선형 또는 평면 함수임을 확인 할 수 있습니다:

![선형 회귀 그래프](https://i.imgur.com/4DJ9f8X.png)

선형 회귀의 _학습_ 은 학습 데이터를 사용하여 가중치 집합 `w11, w12, ... w23, b1 & b2`를 파악하여 새 데이터를 정확하게 예측하는 것입니다.<br>
_학습_ 가중치는 새로운 지역의 <font color=red>평균 온도, 강수량, 습도</font>를 사용해 그 지역의 수확량을 예측하는데 사용됩니다.

저희는 _경사 하강법_ 이라는 최적화 기법을 통해 더 나은 예측을 만들기 위해 모델의 가중치를 여러번 조정해 모델을 훈련시킬 것입니다. <br>
먼저 __Numpy__ 와 __PyTorch__ 를 import하는 것부터 시작합니다.

In [1]:
import numpy as np
import torch

## 훈련 데이터

저희는 `inputs`과 `targets`라는 2개의 행렬을 사용해 훈련 데이터를 표현할 수 있습니다.<br>
각 지역당 행이 1개씩 있고, 변수당 열이 1개씩 있습니다.

In [2]:
# Input (평균 온도, 강수량, 습도)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (사과, 오렌지)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

저희는 input과 target을 따로 사용할것이기에 분리 했습니다. 또한 저희는 일반적인 학습 데이터로 작업하는 방식인 numpy 배렬로 만들었습니다: 일부 CSV파일을 numpy 배열로 읽고 전처리를 한 후 PyTorch 텐서로 변환 합니다.

이제 PyTorch 텐서로 배열을 변환해 봅시다!

In [4]:
# inputs와 targets를 텐서로 변환하기
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## 선형회귀 모델 밑바닥부터 구현해보기

가중치와 편향인 (`w11, w12,... w23, b1 & b2`)는 임의의 값으로 초기화된 행렬로 표현 가능합니다. `w`의 첫 행과 `b`의 첫 요소는 첫 타겟 변수를 예측하기 위해 필요하다. (즉, 사과의 수확량을 예측하는데 사용됨. 두 번째는 오렌지의 수확량을 예측하는데 사용)

In [5]:
# 가중치와 편향
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.1007,  1.2361, -0.3385],
        [-0.3159,  2.9062,  0.0999]], requires_grad=True)
tensor([0.0756, 1.8064], requires_grad=True)


`torch.randn`은 주어진 형태(shape)에 맞추어 평균 0, 표준편차 1인 [정규 분포](https://en.wikipedia.org/wiki/Normal_distribution)에서 임의로 값을 가져와 텐서를 생성합니다. 

저희 *모델*은 단순히 `inputs` 행렬과 가중치 `w`(전치)의 행렬 곱 후 편향인 `b`를 더하는 작업을 수행하게 됩니다. (각 지역마다 복제되서 연산됨)

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

저희는 모델을 다음과 같이 정의 할 수 있습니다:

In [11]:
def model(x):
    return x @ w.t() + b

`@`는 Pytorch의 행렬 곱으로 표현되고, `.t`는 텐서의 전치를 리턴하는 메서드입니다.

입력 데이터를 모델에 전달하여 얻은 행렬은 목표 변수(사과와 오렌지의 수확량)에 대한 예측입니다.

In [12]:
# 예측 생성
preds = model(inputs)
print(preds)

tensor([[ 60.9818, 177.7577],
        [ 78.0167, 235.2001],
        [137.3093, 369.5495],
        [ 30.4271,  98.2481],
        [ 88.0895, 265.9992]], grad_fn=<AddBackward0>)


이제 저희 모델과 실제 값의 차이를 비교해 봅시다.

In [13]:
# targets와 비교
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


저희 모델은 무작위로 가중치와 편향을 초기화했기 때문에 모델의 예측과 실제 값 사이에 큰 차이가 있음을 확인할 수 있습니다.

## 손실 함수

모델을 개선하기 전에, 저희는 모델이 얼마나 잘 작동하는지에 대해 평가할 수 있는 방법이 필요합니다. 저희는 다음과 같은 방법들로 저희 모델의 예측과 실제 목표값을 비교할 수 있습니다:

* `예측`과 `타겟` 행렬의 차이를 계산
* `예측`과 `타겟` 행렬의 차이 행렬의 모든 요소를 제곱해 음수 제거
* 결과 행렬의 요소의 평균을 계산

결과는 **mean squared error** (MSE)로 계산됩니다.

In [14]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum`은 텐서의 모든 요소의 합을 반환합니다. `.numel` 메서드는 텐서의 요소 수를 반환합니다. 이제 현재 우리 모델의 예측의 mean squared error를 계산해 봅시다.

In [15]:
# loss 계산하기
loss = mse(preds, targets)
print(loss)

tensor(11159.9453, grad_fn=<DivBackward0>)


결과를 해석하는 방법: _평균적으로 예측의 각 요소는 손실의 제곱근에 의해 실제 목표와 다릅니다_. 그리고 우리가 예측하려는 숫자가 50 ~ 200 범위에 있다는 점을 생각해보면 매우 안좋습니다. 결과는 모델이 목표 변수를 예측하는데 얼마나 나쁜지를 나타내기 때문에 _loss_ 라고 합니다. _loss_ 는 모델의 정보 손실을 나타내고 이 손실이 적을수록 모델은 더 좋습니다.

## 경사(gradient) 계산하기

Pytorch를 사용하면 가중치와 편향의 `requires_grad`를 `True`로 설정하여 자동으로 가중치와 편향의 기울기(gradient)를 계산할 수 있습니다. 저희는 잠시후 이게 어떻게 유용한지 알아보겠습니다.

In [17]:
# 경사 계산하기
loss.backward()

각 텐서의 경사(gradient)는 `.grad`라는 변수에 저장되어 있습니다. (가중치 행렬의 도함수는 동일한 차원의 행렬이라는 점에 유의)

In [18]:
# 가중치의 경사
print(w)
print(w.grad)

tensor([[-0.1007,  1.2361, -0.3385],
        [-0.3159,  2.9062,  0.0999]], requires_grad=True)
tensor([[  303.1682,   291.1312,    70.6591],
        [11409.7188, 13494.5215,  7899.6777]])


## 손실을 줄이도록 가중치와 편향을 조정하기

손실은 저희의 가중치와 편향의 [2차 함수](https://en.wikipedia.org/wiki/Quadratic_function)입니다. 그리고 저희의 목표는 손실이 가장 낮은 가중치 집합을 찾는 것입니다.
개별 가중치, 편향 요소의 손실 그래프를 그린다면 아래 그림과 같이 보일 것입니다. 미적분학을 사용한 중요한 이해는 경사가 손실의 [변화율](https://en.wikipedia.org/wiki/Slope)을 나타낸다는 것입니다.

만약 경사 요소가 __양__ 일 경우:

* 가중치 요소의 값의 __증가__ 는 손실의 __증가__ 를 일으킵니다.
* 가중치 요소의 값의 __감소__ 는 손실의 __감소__ 를 일으킵니다.


![양의 경사도](https://i.imgur.com/WLzJ4xP.png)

만약 경사 요소가 __음__ 일 경우:

* 가중치 요소의 값의 __증가__ 는 손실의 __감소__ 를 일으킵니다.
* 가중치 요소의 값의 __감소__ 는 손실의 __증가__ 를 일으킵니다.

![음의 경사도](https://i.imgur.com/dvG2fxU.png)

가중치 요소를 변경함으로 인한 손실의 증가 또는 감소는 해당 요소에 대한 손실의 경사와 비례합니다. 이러한 현상은 모델을 향상시키기 위해 저희가 사용하는 _경사 하강_ 최적화 알고리즘에 기반해 형성됩니다.

우리는 손실을 약간 줄이기 위해 각 가중치 요소에서 손실 기울기에 비례하는 작은 값을 뺄 수 있습니다.

In [19]:
w
w.grad

tensor([[  303.1682,   291.1312,    70.6591],
        [11409.7188, 13494.5215,  7899.6777]])

In [20]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

가중치를 많이 수정하지 않기 위해 경사를 매우 작은 수(본 튜토리얼에서는 `10^-5`)를 곱해 빼줍니다. 경사로의 내리막 방향으로 아주 작게 움직이고 싶기 때문입니다. 이 작은 수를 _learning rate_ 라 부릅니다.

가중치와 편향을 수정하는 동안 `torch.no_grad`를 사용해 경사를 추적, 계산, 수정하지 않아야 한다는것을 PyTorch에게 알려줄 수 있습니다.

In [21]:
# 손실의 변화를 확인해보기
loss = mse(preds, targets)
print(loss)

tensor(11159.9453, grad_fn=<DivBackward0>)


계속하기 전, `.zero_()` 메서드를 호출해 경사를 0으로 재설정합니다. PyTorch가 경사를 누적하기 때문에 이 `.zero()`의 호출은 꼭 수행되어야합니다. 그렇지 않으면 다음 번 손실에 대해 `.backward`를 호출시 새로운 경사 값이 기존 경사 값에 추가되어 기대하지 않은 결과가 나올 수 있습니다.

In [17]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## 경사 하강법을 이용한 모델 학습

위에서 봤듯이, 저희는 경사 하강법 알고리즘으로 손실을 줄이고 모델을 향상했습니다. 그러므로, 저희는 다음과 같은 절차로 _학습_ 을 진행 할 수 있습니다:

1. 예측 생성

2. 손실 계산

3. 가중치와 편향의 경사 계산

4. 계산된 경사에 작은 값을 곱한 값을 가중치에서 빼서 가중치 조정

5. 가중치를 0으로 초기화

이제 위의 절차를 구현해봅시다.

In [22]:
# 예측 생성
preds = model(inputs)
print(preds)

tensor([[ 60.5350, 156.9891],
        [ 77.4394, 207.8850],
        [136.6144, 336.9572],
        [ 29.9665,  77.8833],
        [ 87.5514, 239.6406]], grad_fn=<AddBackward0>)


In [23]:
# 손실 계산
loss = mse(preds, targets)
print(loss)

tensor(7746.1274, grad_fn=<DivBackward0>)


In [24]:
# 경사 계산
loss.backward()
print(w.grad)
print(b.grad)

tensor([[  560.3927,   533.1984,   111.0829],
        [20672.7812, 24675.3008, 14373.3203]])
tensor([  4.9862, 249.2220])


위에서 계산한 경사를 이용해 가중치와 편향을 업데이트 합니다.

In [25]:
# 가중치를 업데이트 & 경사 초기화
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

새로운 가중치와 편향을 확인해 봅시다.

In [26]:
print(w)
print(b)

tensor([[-0.1093,  1.2278, -0.3404],
        [-0.6367,  2.5245, -0.1228]], requires_grad=True)
tensor([0.0755, 1.8025], requires_grad=True)


새로운 가중치와 편향으로 모델은 더 작은 손실을 가질 것입니다.

In [27]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(3249.7856, grad_fn=<DivBackward0>)


경사 하강법을 사용해 가중치와 편향을 약간 조절하여 많은 손실을 줄였습니다.

## 여러 epoch동안 학습하기

손실을 더 줄이기 위해, 위에서 진행한 작업을 수차례 반복할 수 있습니다. 이때 각 반복을 _epoch_ 라 부릅니다. 이제 100 epochs 동안 모델을 학습해 봅시다.

In [28]:
# 100 epochs 동안 학습
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

다시 한번, 손실이 작아졌는지 확인해 봅시다.

In [29]:
# 손실 계산
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(283.4501, grad_fn=<DivBackward0>)


손실은 초기에 비해 훨씬 작아졌습니다. 이제 모델의 예측을 실제 목표값과 비교해 봅시다.

In [26]:
# 예측 결과
preds

tensor([[ 60.8975,  70.5663],
        [ 83.9699,  92.9066],
        [108.6802, 150.1993],
        [ 43.5842,  38.4608],
        [ 91.6760, 104.6360]], grad_fn=<AddBackward0>)

In [27]:
# 목표 값
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

이제 예측이 목표 변수에 상당히 근접했습니다. 여기서 약간의 epoch를 더 훈련함해서 훨씬 더 좋은 결과를 얻을 수 있습니다.

## Pytorch에 내장된 선형 회귀

지금까지 선형 회귀와 경사 하강법을 기본 텐서 함수들을 이용해 구현했습니다. 하지만, 이러한 작업들은 딥러닝에서 흔한 패턴이기 때문에 PyTorch는 몇 줄의 코드로 쉽게 생성하고 학습할수 있는 모델들을 여러 내장 함수들과 내장 클래스들로 제공합니다.

이제 신경망을 만들 수 있는 유용한 클래스들을 포함하는 Pytorch의 `torch.nn` 패키지를 import 해 봅시다.

In [30]:
import torch.nn as nn

이전에, 저희는 입력과 목표 값을 행렬로 표현했습니다.

In [31]:
# 입력 (온도, 강우량, 습도)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# 목표 값 (사과, 오렌지)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [32]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

대규모 데이터셋을 소규모 배치로 처리하는 방법을 설명하기 위해 15가지 교육 사례를 사용합니다.

## 데이터셋과 데이터 로더

이제 저희는 `inputs`와 `targets` 튜플에서 행 단위로 접근 가능하고 PyTorch의 다양한 데이터셋에서 작업할 수 있는 표준 API를 제공하는 `TensorDataset`을 만들 것입니다.

In [36]:
from torch.utils.data import TensorDataset

In [37]:
# 데이터셋 정의
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

`TensorDataset`을 사용하면 배열 인덱싱 표기법(위 코드의 `[0:3]`)을 사용해 훈련 데이터의 일부에 접근이 가능합니다. 두 개의 요소를 가진 튜플을 반환하는데 첫 요소에는 선택한 행의 입력 변수가 포함되고 두 번째 요소에는 목표 값이 포함됩니다.

훈련 중에 데이터를 미리 정의된 크기의 배치로 분할할 수 있는 `데이터 로더`도 만들 예정입니다. 데이터 로더는 셔플링 및 데이터의 무작위 샘플링과 같은 기타 유틸리티도 제공합니다.

In [38]:
from torch.utils.data import DataLoader

In [39]:
# 데이터 로더 정의
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

이렇게 생성된 데이터 로더를 `for` 반복문에 사용할 수 있습니다.

In [40]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 68.,  96.,  71.],
        [ 91.,  88.,  64.],
        [ 91.,  87.,  65.],
        [ 87., 135.,  57.],
        [ 69.,  96.,  70.]])
tensor([[104., 118.],
        [ 81., 101.],
        [ 80., 102.],
        [118., 134.],
        [103., 119.]])


각 반복에서 데이터 로더는 지정된 배치 크기의 데이터 배치를 반환합니다. `Shuffle`을 `True`로 설정시 배치 생성 전에 훈련 데이터가 섞입니다. 셔플링은 최적화 알고리즘에 대한 입력을 랜덤화 시키는데 도움이 되므로 손실을 더 빨리 줄일 수 있게 도와줍니다.

## nn.Linear

가중치와 편향을 수동으로 초기화 하는것 대신 PyTorch에서 제공하는 클래스 `nn.Linear`를 사용해 모델을 자동으로 정의 할 수 있습니다.

In [41]:
# 모델 정의
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4850, -0.4457,  0.0572],
        [-0.4896, -0.2447, -0.5019]], requires_grad=True)
Parameter containing:
tensor([ 0.4937, -0.4316], requires_grad=True)


PyTorch의 모델에는 모델에 존재하는 모든 가중치와 편향 행렬을 포함하는 리스트를 반환하는 `.parameters`라는 메서드가 존재합니다. 선형 회귀 모델의 경우 가중치 행렬과 편향 행렬이 하나씩 있습니다.

In [40]:
# 파라미터
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1304, -0.1898,  0.2187],
         [ 0.2360,  0.4139, -0.4540]], requires_grad=True),
 Parameter containing:
 tensor([0.3457, 0.3883], requires_grad=True)]

이전과 같은 방법으로 모델의 예측을 생성할 수 있습니다.

In [41]:
# 예측 생성
preds = model(inputs)
preds

tensor([[ 6.5493, 25.8226],
        [ 9.5025, 29.2272],
        [-1.0633, 50.0460],
        [13.5738, 25.4576],
        [ 6.4278, 24.6221],
        [ 6.8695, 25.6447],
        [ 9.9110, 28.3593],
        [-0.7142, 49.8280],
        [13.2536, 25.6355],
        [ 6.5161, 23.9321],
        [ 6.9578, 24.9546],
        [ 9.8227, 29.0494],
        [-1.4718, 50.9139],
        [13.4855, 26.1476],
        [ 6.1076, 24.8000]], grad_fn=<AddmmBackward>)

## 손실 함수

손실함수를 직접 정의하는 것 대신, 내장 함수인 `mse_loss`를 사용할 수 있습니다.

In [42]:
# Import nn.functional
import torch.nn.functional as F

`nn.functional` 패키지는 많은 유용한 손실 함수와 유틸리티들을 포함합니다.

In [43]:
# Define loss function
loss_fn = F.mse_loss

우리의 모델의 현재 예측에 대한 손실을 계산해 봅시다.

In [44]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(5427.9517, grad_fn=<MseLossBackward>)


## Optimizer

모델의 가중치와 편향의 경사를 사용해 수동으로 수정하는 것 대신, 여기서는 `optim.SGD`를 사용할 것입니다. SGD는 "stochastic gradient descent"의 줄임말로 _stochastic_ 이라는 단어는 샘플이 무작위 배치로 선택되었음을 나타냅니다.

In [1]:
# optimizer 정의
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

NameError: name 'torch' is not defined

`model.parameters()`는 `optim.SGD`에게 넘겨줄 파라미터인것에 유의하세요. 그래야만 optimizer가 어떤 행렬을 수정해야하는지 인지할 수 있습니다. 또한 저희는 파라미터들을 수정시 얼마나 수정할지에 대한 값인 _learning rate_ 또한 명시해줘야 합니다.

## 모델 학습하기

이제 저희는 모델을 학습할 준비가 되었습니다. 저희는 경사 하강법을 다음과 같은 절차로 구현할 수 있습니다:

1. 예측 생성

2. 손실 계산

3. 가중치와 편향의 경사를 계산합니다.

4. 경사에 작은값을 곱한 값을 가중치에 더합니다.

5. 경사를 0으로 재설정합니다.

유일한 변화는 매 반복마다 훈련 데이터를 처리하는 대신 데이터를 배치로 수행한것 입니다. 이제 주어진 Epoch 수에 대해 모델을 훈련시키는 유틸리티 함수 `fit`을 정의 해 보겠습니다.

In [46]:
# 모델을 학습하기 위한 유틸리티 함수
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # epoch의 수만큼 반복합니다.
    for epoch in range(num_epochs):
        
        # 데이터를 batch로 학습
        for xb,yb in train_dl:
            
            # 1. 예측 생성
            pred = model(xb)
            
            # 2. 손실 계산
            loss = loss_fn(pred, yb)
            
            # 3. 경사 계산
            loss.backward()
            
            # 4. 경사를 사용해 파라미터 수정
            opt.step()
            
            # 5. 경사를 0으로 재설정
            opt.zero_grad()
        
        # 현재 상태를 출력
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

위의 코드의 참고 사항:

* 앞에서 정의한 데이터 로더를 사용해 모든 반복에 대한 데이터 배치를 가져옵니다.

* 매개변수(가중치 & 편향)을 수동으로 업데이트 하는 대신 `opt.step`을 사용해 업데이트를 하였고, `opt.zero_grad`로 경사를 0으로 만들었다.

* 또한 교육 진행률을 추적하기 위해 10 epoch마다 마지막 데이터 배치의 손실을 인쇄하는 로그 문장을 추가했습니다. `loss.item`은 손실 텐서에 저장된 실제 손실 값을 반환합니다.

이제 모델을 100 epoch 동안 학습해 봅시다.

In [47]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 818.6476
Epoch [20/100], Loss: 335.3347
Epoch [30/100], Loss: 190.3544
Epoch [40/100], Loss: 131.6701
Epoch [50/100], Loss: 77.0783
Epoch [60/100], Loss: 151.5671
Epoch [70/100], Loss: 151.0817
Epoch [80/100], Loss: 67.6262
Epoch [90/100], Loss: 53.6205
Epoch [100/100], Loss: 33.4517


모델을 사용해 예측을 생성하고 목표에 가까운지 확인하겠습니다.

In [48]:
# 예측 생성
preds = model(inputs)
preds

tensor([[ 58.4229,  72.0145],
        [ 82.1525,  95.1376],
        [115.8955, 142.6296],
        [ 28.6805,  46.0115],
        [ 97.5243, 104.3522],
        [ 57.3792,  70.9543],
        [ 81.9342,  94.1737],
        [116.2036, 142.6871],
        [ 29.7242,  47.0717],
        [ 98.3498, 104.4486],
        [ 58.2047,  71.0507],
        [ 81.1088,  94.0774],
        [116.1137, 143.5935],
        [ 27.8550,  45.9152],
        [ 98.5680, 105.4124]], grad_fn=<AddmmBackward>)

In [49]:
# 목표와 비교
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

사실, 예측은 목표 값과 꽤 근접했습니다. 지금까지 한 지역의 평균 기온, 강우량 및 습도를 조사하여 사과와 오렌지의 작물 수확량을 예측하는 좋은 모델을 훈련 시켰습니다. 입력 한 줄을 포함하는 배치를 전달해 새로운 지역의 농작물 수확량을 예측하는데 모델을 사용할 수 있습니다.

In [50]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[55.3323, 67.8895]], grad_fn=<AddmmBackward>)

예측된 수확량은 1 헥타르당 54.3 톤 이고 오렌지는 1 헥타르당 68.3 톤입니다.

## 기계 학습 vs 클래식 프로그래밍

이 튜토리얼의 접근 방식은 기존에 저희가 알던 프로그래밍과 많이 다릅니다. 보통, 저희는 몇가지 입력이 필요하고 몇가지 연산을 하고 결과를 반환하는 프로그램을 작성합니다.

하지만, 이 튜토리얼에선, 일부 알려지지 않은 매개변수(추정 & 편견)을 사용하여 표현된 입력과 출력 사이의 특정 관계를 가정하는 "모델"을 정의 했습니다. 우리는 정의한 모델에게 입력 및 출력을 알고있는 모델을 보여주고 알려지지 않은 매개 변수에 대한 좋은 값을 제시하도록 모델을 _훈련_ 합니다. 한번 훈련이 되었다면, 그 모델은 새로운 input들에 대한 결과를 계산하는데 사용될 수 있습니다.

이 프로그래밍 패러다임은 _기계 학습_ 이라고 하는데, 데이터를 사용해 입력과 출력 사이의 관계를 파악합니다. _딥 러닝_ 은 기계학습의 한 갈래로 행렬 연산, 비 선형 활성화 함수, 경사 하강법 등 모델을 만들고 학습하는 방법들을 사용합니다. 테슬라 자동차의 인공지능 디렉터 Andrej Karpathy는 [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35)라는 제목의 굉장히 좋은 글을 남겼습니다. 시간이 되신다면 읽어보시기 바랍니다.

아래 그림은 [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python)이라는 책에서 가져온 기계학습과 고전 프로그래밍의 차이를 보여주는 그림입니다.

![](https://i.imgur.com/oJEQe7k.png)

다음 튜토리얼들을 하면서 이 그림을 꼭 마음속에 염두해두고 하시길 바랍니다.

## 연습 문제와 추가 읽을거리

저희는 이번 튜토리얼에서 다음과 같은 주제들을 다뤘습니다.

- 선형 회귀와 경사 하강법의 소개
- PyTorch 텐서를 이용해 선형 회귀 구현하기
- 선형 회귀 모델을 경사 하강법 알고리즘을 통해 훈련시키기
- PyTorch에 내장된 경사 하강법과 선형 회귀를 이용해 앞에서 다룬 내용 구현해보기

선형 회귀와 경사 하강법에 더 공부해보고 싶으시다면 다음 자료들을 읽어보시기 바랍니다:

* 경사 하강법을 시각적 & 영상화하여 설명한 자료 : https://www.youtube.com/watch?v=IHZwWFHWa-w
 
* 미분과 경사 하강법에 대한 더욱 자세한 설명은 [Udacity course의 노트](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf)를 보십시오.

* 선형 회귀가 어떻게 작동하는지에 대한 동영상은 [이 포스트를 보시기 바랍니다](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* 행렬 미적분, 선형 회귀, 경사 하강법에 대한 수학적 이해에 대해서는 스탠포드의 [Andrew Ng's excellent course note](https://github.com/Cleo-Stanford-CS/CS229_Notes/blob/master/lectures/cs229-notes1.pdf)를 보시기 바랍니다.

* 여러분의 스킬을 연습 및 테스트 해보려면 캐글에서 열린 대회인 [보스턴 집 값 예측 대회](https://www.kaggle.com/c/boston-housing)를 해보십시오. (캐글에서는 이런 데이터과학 대회가 개최되므로 참여해 보시는걸 추천합니다.)

지금까지, 저희는 PyTorch에서의 선형 회귀에 대해 알아보았습니다. 다음 튜토리얼에선 PyTorch에서의 이미지와 로지스틱 회귀 분석 작업에 대해 알아보겠습니다.

## Questions for Review

Try answering the following questions to test your understanding of the topics covered in this notebook:

1. What is a linear regression model? Give an example of a problem formulated as a linear regression model.
2. What are input and target variables in a dataset? Give an example.
3. What are weights and biases in a linear regression model?
4. How do you represent tabular data using PyTorch tensors?
5. Why do we create separate matrices for inputs and targets while training a linear regression model?
6. How do you determine the shape of the weights matrix & bias vector given some training data?
7. How do you create randomly initialized weights & biases with a given shape?
8. How is a linear regression model implemented using matrix operations? Explain with an example.
9. How do you generate predictions using a linear regression model?
10. Why are the predictions of a randomly initialized model different from the actual targets?
11. What is a loss function? What does the term “loss” signify?
12. What is mean squared error?
13. Write a function to calculate mean squared using model predictions and actual targets.
14. What happens when you invoke the `.backward` function on the result of the mean squared error loss function?
15. Why is the derivative of the loss w.r.t. the weights matrix itself a matrix? What do its elements represent?
16. How is the derivate of the loss w.r.t. a weight element useful for reducing the loss? Explain with an example.
17. Suppose the derivative  of the loss w.r.t. a weight element is positive. Should you increase or decrease the element’s value slightly to get a lower loss?
18. Suppose the derivative  of the loss w.r.t. a weight element is negative. Should you increase or decrease the element’s value slightly to get a lower loss?
19. How do you update the weights and biases of a model using their respective gradients to reduce the loss slightly?
20. What is the gradient descent optimization algorithm? Why is it called “gradient descent”?
21. Why do you subtract a “small quantity” proportional to the gradient from the weights & biases, not the actual gradient itself?
22. What is learning rate? Why is it important?
23. What is `torch.no_grad`?
24. Why do you reset gradients to zero after updating weights and biases?
25. What are the steps involved in training a linear regression model using gradient descent?
26. What is an epoch?
27. What is the benefit of training a model for multiple epochs?
28. How do you make predictions using a trained model?
29. What should you do if your model’s loss doesn’t decrease while training? Hint: learning rate.
30. What is `torch.nn`?
31. What is the purpose of the `TensorDataset` class in PyTorch? Give an example.
32. What is a data loader in PyTorch? Give an example.
33. How do you use a data loader to retrieve batches of data?
34. What are the benefits of shuffling the training data before creating batches?
35. What is the benefit of training in small batches instead of training with the entire dataset?
36. What is the purpose of the `nn.Linear` class in PyTorch? Give an example.
37. How do you see the weights and biases of a `nn.Linear` model?
38. What is the purpose of the `torch.nn.functional` module?
39. How do you compute mean squared error loss using a PyTorch built-in function?
40. What is an optimizer in PyTorch?
41. What is `torch.optim.SGD`? What does SGD stand for?
42. What are the inputs to a PyTorch optimizer? 
43. Give an example of creating an optimizer for training a linear regression model.
44. Write a function to train a `nn.Linear` model in batches using gradient descent.
45. How do you use a linear regression model to make predictions on previously unseen data? 

